In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [2]:
cps = pd.read_csv("data/cps.csv.gz")
cps.columns = cps.columns.str.lower()

In [3]:
cps.columns

Index(['year', 'serial', 'month', 'hwtfinl', 'cpsid', 'asecflag', 'asecwth',
       'statefip', 'pernum', 'wtfinl', 'cpsidp', 'asecwt', 'age', 'sex',
       'race', 'marst', 'citizen', 'hispan', 'empstat', 'educ', 'inctot',
       'adjginc', 'spmtotres', 'spmthresh', 'spmfamunit', 'voted', 'vosuppwt'],
      dtype='object')

In [4]:
cps["vote"] = cps.voted == 2

In [6]:
p = cps[cps.month == 3].copy()
train = cps[cps.month == 11].sample(frac=1.0).copy()
train["vote"] = train.voted == 2

In [7]:
XCOLS = ["statefip", 'age', 'sex', 'race', 'marst', 'citizen', 'hispan', 'empstat', 'educ',]
rf = RandomForestClassifier().fit(train[XCOLS], train.vote, train.vosuppwt)

KeyboardInterrupt: 

In [ ]:
preds = rf.predict_proba(p[XCOLS])
p["pred_vote"] = preds[:, 1]

In [ ]:
cps[cps.month == 11].groupby("year").vote.mean()

In [ ]:
p.pred_vote.mean()

In [ ]:
p[["age", "educ", "citizen", "pred_vote"]].sample(10)

In [ ]:
# Check that 18 year olds have more than zero.
p[p.age == 18][["age", "educ", "citizen", "pred_vote"]]

In [ ]:
pd.Series(rf.feature_importances_, index=XCOLS).sort_values(ascending=False)

In [ ]:
p.to_csv("data/asec_vote_prob.csv.gz", index=False, compression="gzip")

* Add FAMINC
* Add wage
* Add 2018-2019 ASEC
* Share of population comes out ahead
* Share of eligibleadults citizens
* Share of predicted voters
* Share of predicted voters by state (slice by 2022)
* Each for a 2x2 of policy designs: include kids vs. include non-citizens
* Add context of other polls and poverty/inequality effects
